In [50]:
%%bash
pip install datalab
pip install tensorflow-gpu

  Running setup.py bdist_wheel for datalab: started
  Running setup.py bdist_wheel for datalab: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a0/49/91/b042b0fc2f6f1b418c0d4adc9aee43af3e1b6784367c6700df
  Running setup.py bdist_wheel for configparser: started
  Running setup.py bdist_wheel for configparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c
Successfully built datalab configparser
  Found existing installation: six 1.11.0
    Uninstalling six-1.11.0:
      Successfully uninstalled six-1.11.0


spacy 2.0.17 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 0.0.1a1 has requirement six~=1.11.0, but you'll have six 1.10.0 which is incompatible.
tcmalloc: large alloc 1073750016 bytes == 0x50c00000 @  0x7f601bbdb2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [51]:
#Import required libraries
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

# from keras.models import Sequential
# from nltk.tokenize import word_tokenize
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.naive_bayes import GaussianNB
# from xgboost.sklearn import XGBClassifier
# https://machinelearningmastery.com/clean-text-machine-learning-python/
print(tf.__version__)

1.12.0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [52]:
#Check available devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6625221293321700420
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 12717716568918420733
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6945412640120490886
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17914494300379395380
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [53]:
#Load training data
all_data = pd.read_csv('drive/My Drive/Colab Notebooks/train_data.csv')
all_data.head()

,text,label
0,never talk to me again,3
1,I am proud of your achievements,2
2,It is the worst day in my life,3
3,Miss you so much,0
4,food is life,4


In [0]:
#Create one hot encoding of labels
def create_yoh_list(v_classes, num_classes):
    yoh_class = np.zeros((len(v_classes),num_classes))
    
    for ix, v_cl in enumerate(v_classes):
        yoh_class[ix, v_cl] = 1
    
    return(yoh_class)

In [55]:
#Test one hot encoder code
create_yoh_list([1,3,4,2],num_classes = 5)

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.]])

In [56]:
#Def class to text
def class_to_text(cls):
    dict_class = {0:'love', 1:'playful', 2:'happy',3:'sad', 4:'foodie'}
    return dict_class[cls]

class_to_text(2)

'happy'

In [67]:
#Random split dataset into Train and Validation
train_data = all_data.sample(frac=0.95,random_state=1)
valid_data = all_data.drop(train_data.index)

train_data.info()
valid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 31 to 134
Data columns (total 2 columns):
text     179 non-null object
label    179 non-null int64
dtypes: int64(1), object(1)
memory usage: 4.2+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9 entries, 37 to 144
Data columns (total 2 columns):
text     9 non-null object
label    9 non-null int64
dtypes: int64(1), object(1)
memory usage: 216.0+ bytes


In [0]:
# Define your feature columns
def create_feature_cols():
    embedded_text_feature_column = hub.text_embedding_column(key="text", 
                                    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1",
                                    trainable=False)
    return [embedded_text_feature_column]

In [0]:
# Create pandas input function for training
def make_input_fn(df_f, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df_f,
    y = df_f['label'],
    batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 200,
    num_threads = 1
  )

#Input Function for Train and Validation Set
def get_validation_input_fn(data_set, num_epochs=1, shuffle=False):
  return tf.estimator.inputs.pandas_input_fn(
      x = data_set,
      y = data_set['label'],
      num_epochs=num_epochs,
      shuffle=shuffle)

# Serving function for external call
def serving_fn():
    feature_placeholders  = {'text' : tf.placeholder(tf.string, [None])
    }

    #Features with transformation logic
    features = {
                key: tf.expand_dims(tensor, -1)
                for key, tensor in feature_placeholders.items()
            }
    
    #feat_changed = add_engineered(features.copy())
    return tf.estimator.export.ServingInputReceiver(features, feature_placeholders )

In [0]:
def add_engineered(inp):
    pass

In [0]:
# Create estimator train and evaluate function
def train_and_evaluate(output_dir, num_train_steps):    
##### Create Canned estimator instance
    run_config = tf.estimator.RunConfig(save_checkpoints_secs = 40, 
                                        keep_checkpoint_max = 3)
    estimator = tf.estimator.DNNClassifier(feature_columns=create_feature_cols(),
                                          n_classes=5,
                                          hidden_units=[256,128,64,32],
                                          dropout = 0.3,
                                          #optimizer=get_stepw_decay_optimizer)
                                          optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
                                          config = run_config)
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(train_data, None), 
                                      max_steps = num_train_steps)
    exp = tf.estimator.LatestExporter("decision", serving_fn)
    eval_spec = tf.estimator.EvalSpec(input_fn = get_validation_input_fn(valid_data, 1), 
                                    steps = None, 
                                    exporters = exp,
                                    start_delay_secs = 1, # start evaluating after N seconds, 
                                    throttle_secs = 20)  # evaluate every N seconds
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

In [73]:
train_and_evaluate(None, num_train_steps=10000)

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9mzo_fdl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 40, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fee1716a160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The eval

In [0]:
from google.datalab.ml import TensorBoard
OUTDIR='/tmp/tmp5qwzs4y8'
print(OUTDIR)
TensorBoard().start(OUTDIR)